<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/classifier101_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai h5py pillow
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4
import numpy as np
import pandas as pd
import os
from fastai.vision.all import *

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/kaggle/isic-2024-data/

random_forest_model.pkl  test-image.hdf5    test-metadata.gsheet  train-metadata.csv
sample_submission.csv	 test-metadata.csv  train-image.hdf5	  train-metadata-v1.csv


In [4]:
HDF5_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-image.hdf5"
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"

In [5]:
import pandas as pd

# Load the metadata CSV file
metadata_df = pd.read_csv(METADATA_FILE)
# Display the first few rows to understand the structure
metadata_df = metadata_df[:90000]
metadata_df[:100]


<ipython-input-5-68bf84badc25>:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ISIC_0074664,0,IP_9179679,70.0,male,posterior torso,3.01,TBP tile: close-up,3D: XP,21.384960,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.991430
96,ISIC_0074666,0,IP_7295514,60.0,male,posterior torso,2.91,TBP tile: close-up,3D: XP,25.651020,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999930
97,ISIC_0075664,0,IP_4825600,60.0,male,posterior torso,5.80,TBP tile: close-up,3D: XP,19.594920,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.984705
98,ISIC_0075676,0,IP_5092630,70.0,female,lower extremity,2.55,TBP tile: close-up,3D: white,20.130138,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.995089


In [6]:
import numpy as np
from PIL import Image
import io

# Function to extract features from image data
def extract_features(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((128, 128))  # Resize image to 128x128 pixels
    return np.array(image).flatten()  # Flatten the image into a 1D array


In [7]:
from sklearn.model_selection import train_test_split

# Split the metadata DataFrame into training and testing sets
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

# Extract the 'isic_id' and 'target' columns for later use
ids_train = train_df['isic_id'].values
y_train = train_df['target'].values
ids_test = test_df['isic_id'].values
y_test = test_df['target'].values


In [8]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data



In [ ]:
import h5py

# Initialize lists to hold the entire training data
X_train = []
y_train = []

# Open the HDF5 file and process the training data in batches
with h5py.File(HDF5_FILE, 'r') as hdf:
    for df_batch in np.array_split(train_df, 10):  # Split into smaller batches
        X_batch = []
        y_batch = []

        for _, row in df_batch.iterrows():
            image_id = row['isic_id']

            if image_id in hdf:
                # Read the image data
                image_data = hdf[image_id][()]

                # Extract image features
                features = extract_features(image_data)

                # Append the features and corresponding label
                X_batch.append(features)
                y_batch.append(row['target'])

        # Convert batch lists to NumPy arrays
        X_batch = np.array(X_batch)
        y_batch = np.array(y_batch)

        # Append batch data to the overall training data
        X_train.extend(X_batch)
        y_train.extend(y_batch)

# Convert the final lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Train the RandomForest model on the entire training data
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
import h5py

# Initialize lists to hold the test data
X_test = []

# Open the HDF5 file and process the test data
with h5py.File(HDF5_FILE, 'r') as hdf:
    for image_id in ids_test:
        if image_id in hdf:
            image_data = hdf[image_id][()]
            features = extract_features(image_data)
            X_test.append(features)

# Convert lists to NumPy arrays for evaluation
X_test = np.array(X_test)

# Predict probabilities for the test set
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probability of being malignant


In [ ]:
# Create a DataFrame for the predictions
predictions_df = pd.DataFrame({
    'isic_id': ids_test,
    'target': y_pred_prob
})

# Save the predictions to a CSV file
predictions_df #.to_csv('predictions.csv', index=False, header=True)


In [ ]:
import joblib

# Save the trained model to a file
joblib_filename = 'random_forest_model.joblib'
joblib.dump(model, joblib_filename)


In [ ]:
import pickle

# Save the trained model to a file
pickle_filename = '/content/drive/MyDrive/kaggle/isic-2024-data/random_forest_model.pkl'
with open(pickle_filename, 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import pickle

# Load the model from the pkl file
with open(pickle_filename, 'rb') as file:
    model = pickle.load(file)


In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC

    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC)
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(np.asarray(solution.values)-1)

    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)
    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)

    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)

    partial_auc = auc(fpr, tpr)

    return(partial_auc)

solution = test_df[['isic_id', 'target']]
predictions = predictions_df[['isic_id', 'target']]

partial_auc = score(solution, predictions, "isic_id")
partial_auc